<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/main/Webscraper3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
from bs4 import BeautifulSoup as b4
import asyncio
import multiprocessing
import time
import requests
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [59]:
TRIAL = 3

### Parse chapter

In [88]:
def parseFile(html):
  #with open(html, "r") as file:
  soup = b4(html, features="html.parser")
  #print(soup.chapter-title.get_text())
  tag = soup.article.find_all('span')
  content = ""
  for t in tag:
    if "Chapter" in t.get_text():
      title = t.get_text() + ".html"
  soup = soup.find_all('p')
  with open(title, "w" , encoding='utf-8') as file:
    for paragraph in soup:
        file.write(str(paragraph) + "\n")  # Write each paragraph to the file
    print(str(soup))
  file.close()

### Download main webpage

In [62]:
def getChapter(url, browser, trial):
  try:
      browser.get(url)
      timeout_in_seconds = 10
      WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
      html = browser.page_source
      parseFile(html)
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        file.close()
        print("I give up...")
      else:
        getChapter(url, browser, trial + 1)
  finally:
      browser.quit()

### Get webpage

In [89]:
def getNovelPage(url, browser, trial):
  try:
      browser.get(url)
      html = browser.page_source
      soup = b4(html, features="html.parser")
      strainer = soup.find(class_="chapter-list").find_all('a')
      for link in strainer:
        new_url = url.replace("/chapters", "") + link['href']
        getChapter(new_url, browser, 0)
      return html
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        return -1
        file.close()
        print("I give up...")
      else:
        getNovelPage(url, browser, trial + 1)
  finally:
      browser.quit()

### Get links and go to next chapter page, if avaliable (testing from file)

In [92]:
def getChapterPages(url, page_num):
  # Start a new selenium process
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  browser = webdriver.Firefox(options=options)

  # Get the novel main page
  url_source = getNovelPage(url, browser, 0)

  if url_source == -1:
    print("Unable to find page source!")
    return

  soup = b4(url_source, features="html.parser")
  links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")
  next_url = ""
  for link in links:
    next_url = link.find('a')['href']

  # Make new url
  new_url = url.replace("/chapters", "") + next_url
  getChapterPages(new_url, page_num+1)

### Input url

In [ ]:
url = input("Enter URL: ")
#sample url: https://www.lightnovelpub.com/novel/chrysalis-16091353/chapters
getChapterPages(url, 1)

Enter URL: https://www.lightnovelpub.com/novel/chrysalis-16091353/chapters


## Test